---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [29]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [30]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
states

{'OH': 'Ohio',
 'KY': 'Kentucky',
 'AS': 'American Samoa',
 'NV': 'Nevada',
 'WY': 'Wyoming',
 'NA': 'National',
 'AL': 'Alabama',
 'MD': 'Maryland',
 'AK': 'Alaska',
 'UT': 'Utah',
 'OR': 'Oregon',
 'MT': 'Montana',
 'IL': 'Illinois',
 'TN': 'Tennessee',
 'DC': 'District of Columbia',
 'VT': 'Vermont',
 'ID': 'Idaho',
 'AR': 'Arkansas',
 'ME': 'Maine',
 'WA': 'Washington',
 'HI': 'Hawaii',
 'WI': 'Wisconsin',
 'MI': 'Michigan',
 'IN': 'Indiana',
 'NJ': 'New Jersey',
 'AZ': 'Arizona',
 'GU': 'Guam',
 'MS': 'Mississippi',
 'PR': 'Puerto Rico',
 'NC': 'North Carolina',
 'TX': 'Texas',
 'SD': 'South Dakota',
 'MP': 'Northern Mariana Islands',
 'IA': 'Iowa',
 'MO': 'Missouri',
 'CT': 'Connecticut',
 'WV': 'West Virginia',
 'SC': 'South Carolina',
 'LA': 'Louisiana',
 'KS': 'Kansas',
 'NY': 'New York',
 'NE': 'Nebraska',
 'OK': 'Oklahoma',
 'FL': 'Florida',
 'CA': 'California',
 'CO': 'Colorado',
 'PA': 'Pennsylvania',
 'DE': 'Delaware',
 'NM': 'New Mexico',
 'RI': 'Rhode Island',
 'MN': 'M

In [31]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
  
      # First we read the txt file, we have to use new line (\n) as separator. 
    university_towns = pd.read_csv('university_towns.txt', sep='\n', header = None )
    
    # Then we clean the data by removing parenthesis and squared brackets
    # expand = True splits the string into different columns.
    for col in university_towns:
        university_towns[col] = university_towns[col].str.split('(',expand=True)[0].str.split('[', expand=True)[0].str.rstrip()
     
    
      # We have to find now the states in the data, to use them later to create a Series with them, 
      # which will be the first column with the States names. To do so, first get them from the values in 
      # the given dictionary (states), then we sort alphabetically the states (because they appear alphabetically in 
      # our df university_towns) and we remove those states which are not in our dataframe (because these states do not have uni towns).
    stts = sorted(list(states.values()))
    for st in stts:
        if st not in university_towns.iloc[:,0].tolist():
            stts.remove(st)
    
      # Now it is time to create the list (States) which will contain the States repeated as many times as uni towns they have. 
      # To do so we iterate through a for loop and we get the index of the next state in university_towns.iloc[:,0] (n+1) and
      # substract it from the index of the State we are checking (n). The result of this substraction is the number of uni towns 
      # from the n-State (+1 extra for the State itself, which is not a town, so we will have to remove those extra lines later)
    States = []
    for n in range(0,len(stts)-1):
        i = university_towns.index[university_towns.iloc[:,0]==stts[n]].tolist()[0]
        j = university_towns.index[university_towns.iloc[:,0]==stts[n+1]].tolist()[0]
        x=j-i
        States = States + list([stts[n]]*x)
      # There is a problem in the loop with the last State, so we have to use this trick to complete the States list. 
      # Extra is the difference between the length of the df column university_towns.iloc[:,0] and the length of Sates which 
      # is missing the last State so far. This gives us the number of times the last State has to be repeated and we add it to 
      # the States list to complete it
    extra = len(university_towns.iloc[:,0].tolist())-len(States)
    States = States + list([stts[-1]]*extra)

      # We convert now the States list to a Series, add it to our df, rename the columns and finally reorder it
    university_towns['State'] = pd.Series(States)
    university_towns.columns = ["RegionName", "State"]
    university_towns = university_towns[['State', "RegionName"]]

      # Remember we have extra lines, where the States appear as towns. We have to remove those, by using the drop function
    university_towns.drop(university_towns[university_towns["State"]==university_towns["RegionName"]].index, inplace = True)
    university_towns.reset_index(inplace=True,drop=True)
    
    return university_towns

get_list_of_university_towns()
university_towns = pd.read_csv('university_towns.txt', sep='\n', header = None )
for col in university_towns:
        university_towns[col] = university_towns[col].str.split('(',expand=True)[0].str.split('[', expand=True)[0].str.rstrip()

university_towns

,0
0,Alabama
1,Auburn
2,Florence
3,Jacksonville
4,Livingston
...,...
562,Stevens Point
563,Waukesha
564,Whitewater
565,Wyoming


In [32]:
def get_recession_start():
        # We read the gdp excel, skipping the columns and rows we will not use. We rename the columns too with more meaningful names
    gdp = pd.read_excel('gdplev.xls', header = None, skiprows = range(0, 8), usecols = [4,5,6])
    gdp.columns = ['Quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars']

        # We will use data from the first Quarter of 2000 on, so we need the index of this row, and then we use the dataframe from 
        # this row on. We also need to reset the row index to have it beginning at 0
    gdp.index[gdp['Quarter']=='2000q1'].tolist()[0]
    gdp = gdp.iloc[gdp.index[gdp['Quarter']=='2000q1'].tolist()[0]:,:]
    gdp.reset_index(drop=True, inplace=True)

        # We create another column to check the porcentual variation of gdP
    gdp['GDP in billions of chained 2009 dollars variation (%)'] = gdp.loc[:,'GDP in billions of chained 2009 dollars'].pct_change() * 100

        # With a for loop we check the row where the recession starts (a recession starts when there are two consecutive quarters 
        # of GDP decline)
    recession_index = []
    for n in range(0,len(gdp.loc[:,'GDP in billions of chained 2009 dollars variation (%)'].tolist())-1):
        if (gdp.loc[n,'GDP in billions of chained 2009 dollars variation (%)']<0) & (gdp.loc[n+1,'GDP in billions of chained 2009 dollars variation (%)']<0):
            recession_index.append(n)

        # If there are several consecutive quarters with negative porcentual variation, we just need the one when it started, so we remove the following ones
        # With the first for loop we check the consecutive indexes and we create a list (to_remove) which we will use later in another loop to obtain just the 
        # quarter when the recession started as explained before
    to_remove = []
    for n in range(0,2):
        if recession_index[n+1]==recession_index[n]+1:
            to_remove.append(recession_index[n+1])
    recession_index = [x for x in recession_index if x not in to_remove]

        # We can now obtain the qurter when the recssion started
    return gdp.iloc[recession_index[0],0]
    
get_recession_start()


'2008q3'

In [33]:
def get_recession_end():
    
    '''Returns the year and quarter of the recession end as a 
    string value in a format such as 2005q3'''
    
    # ----------------------------------------------------------------------------------------------------------------------------------------------
    # -------------------- WE WILL REUSE PART OF THE CODE FROM THE PREVIOUS FUNTION -------------------------------------------------------------------
    # ----------------------------------------------------------------------------------------------------------------------------------------------
    
    # We read the gdp excel, skipping the columns and rows we will not use. We rename the columns too with more meaningful names
    gdp = pd.read_excel('gdplev.xls', header = None, skiprows = range(0, 8), usecols = [4,5,6])
    gdp.columns = ['Quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars']
        # We will use data from the first Quarter of 2000 on, so we need the index of this row, and then use the dataframe from 
        # this row on. We also need to reset the row index to have it beginning at 0
    gdp.index[gdp['Quarter']=='2000q1'].tolist()[0]
    gdp = gdp.iloc[gdp.index[gdp['Quarter']=='2000q1'].tolist()[0]:,:]
    gdp.reset_index(drop=True, inplace=True)

        # We create another column to check the porcentual variation of gdP
    gdp['GDP in billions of chained 2009 dollars variation (%)'] = gdp.loc[:,'GDP in billions of chained 2009 dollars'].pct_change() * 100

        # With a for loop we check the row where the recession starts (a recession starts when there are two consecutive quarters 
        # of GDP decline)
    recession_index = []
    for n in range(0,len(gdp.loc[:,'GDP in billions of chained 2009 dollars variation (%)'].tolist())-1):
        if (gdp.loc[n,'GDP in billions of chained 2009 dollars variation (%)']<0) & (gdp.loc[n+1,'GDP in billions of chained 2009 dollars variation (%)']<0):
            recession_index.append(n)
    
    # ----------------------------------------------------------------------------------------------------------------------------------------------
    # -----------------------------HERE FINISHES THE CODE REUSE----------------------------------------------------------------------------------------
    # ----------------------------------------------------------------------------------------------------------------------------------------------
        
        # We use the same logic in the for loop to find the row where the recession ends (a recession ends when there are two consecutive quarters 
        # of GDP increase). In this case we need to break the loop after finding the row
    end_of_recession_index = []
    for n in range(recession_index[0],len(gdp.loc[:,'GDP in billions of chained 2009 dollars variation (%)'].tolist())-1):
        if (gdp.loc[n,'GDP in billions of chained 2009 dollars variation (%)']>0) & (gdp.loc[n+1,'GDP in billions of chained 2009 dollars variation (%)']>0):
            end_of_recession_index.append(n+1)
            break

        # We can now obtain the qurter when the recssion finished
    return gdp.iloc[end_of_recession_index[0],0]
    
get_recession_end()

'2009q4'

In [34]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
        
    # ----------------------------------------------------------------------------------------------------------------------------------------------
    # -------------------- WE WILL REUSE PART OF THE CODE FROM THE PREVIOUS FUNTION -------------------------------------------------------------------
    # ----------------------------------------------------------------------------------------------------------------------------------------------
    
        # We read the gdp excel, skipping the columns and rows we will not use. We rename the columns too with more meaningful names
    gdp = pd.read_excel('gdplev.xls', header = None, skiprows = range(0, 8), usecols = [4,5,6])
    gdp.columns = ['Quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars']
        # We will use data from the first Quarter of 2000 on, so we need the index of this row, and then use the dataframe from 
        # this row on. We also need to reset the row index to have it beginning at 0
    gdp.index[gdp['Quarter']=='2000q1'].tolist()[0]
    gdp = gdp.iloc[gdp.index[gdp['Quarter']=='2000q1'].tolist()[0]:,:]
    gdp.reset_index(drop=True, inplace=True)

        # We create another column to check the porcentual variation of gdP
    gdp['GDP in billions of chained 2009 dollars variation (%)'] = gdp.loc[:,'GDP in billions of chained 2009 dollars'].pct_change() * 100

        # With a for loop we check the row where the recession starts (a recession starts when there are two consecutive quarters 
        # of GDP decline)
    recession_index = []
    for n in range(0,len(gdp.loc[:,'GDP in billions of chained 2009 dollars variation (%)'].tolist())-1):
        if (gdp.loc[n,'GDP in billions of chained 2009 dollars variation (%)']<0) & (gdp.loc[n+1,'GDP in billions of chained 2009 dollars variation (%)']<0):
            recession_index.append(n)
        # With a for loop we check the row where the recession starts (a recession starts when there are two consecutive quarters 
        # of GDP decline)
    end_of_recession_index = []
    for n in range(recession_index[0],len(gdp.loc[:,'GDP in billions of chained 2009 dollars variation (%)'].tolist())-1):
        if (gdp.loc[n,'GDP in billions of chained 2009 dollars variation (%)']>0) & (gdp.loc[n+1,'GDP in billions of chained 2009 dollars variation (%)']>0):
            end_of_recession_index.append(n+1)
            break
    
    # ----------------------------------------------------------------------------------------------------------------------------------------------
    # -----------------------------HERE FINISHES THE CODE REUSE----------------------------------------------------------------------------------------
    # ----------------------------------------------------------------------------------------------------------------------------------------------
         
        # Now we reduce the gdp df to the recession rows, we find the minimum of GDP in billions of chained 2009 dollars within
        # this period, the row index for this minimum and finally the quarter when it happened
    recession_df = gdp.iloc[recession_index[0]:end_of_recession_index[0]+1,:]
    min_GDP_in_recession = recession_df['GDP in billions of chained 2009 dollars'].min()
    recession_bottom_index = gdp.index[gdp['GDP in billions of chained 2009 dollars']==min_GDP_in_recession].tolist()[0]
    gdp.iloc[recession_bottom_index,0]
    return gdp.iloc[recession_bottom_index,0]

get_recession_bottom()

'2009q2'

In [41]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    house_data = pd.read_csv('City_Zhvi_AllHomes.txt')
    # We delete the months we are not interested in (before 2000-01)
    house_data.drop(house_data.iloc[:,6:51], axis=1, inplace=True)
    # Now we calculate the means of the quarters
    for i in range(6,203,3):
        house_data.iloc[:,i] = house_data.iloc[:,[i,i+1,i+2]].mean(axis=1)
    # We only have two columns for the last quarter, so we calculate it with these two
    house_data.iloc[:,-2] = house_data.iloc[:,[-1,-2]].mean(axis=1)
    # We need to get rid now of those columns which do not contain the means (the quarters)
    house_data.drop(house_data.iloc[:,7:206:3], axis=1, inplace=True)
    house_data.drop(house_data.iloc[:,7:138:2], axis=1, inplace=True)
    # Now we map the two letter code for the states to complete names using the dict 'States' given at the very beginning 
    # of the project 
    house_data['State'] = house_data['State'].map(states)
    # Now we set the multiindex as requested in the exercise
    house_data.set_index(['State', 'RegionName'], inplace=True)
    house_data.sort_index(inplace=True)
    # We get rid of the columns which are not necessary (RegionID, Metro, CountyName, SizeRank)
    house_data.drop(house_data.iloc[:,0:4], axis=1, inplace=True)
    # We need to rename the columns using the quarters notation. To do so we create a list:
    columns = []
    for i in range(2000,2017):
        for j in range(1,5):
            columnname = str(i)+ 'q' + str(j)
            columns.append(columnname)
    # We have an extra column name (2016q4) which we need to remove 
    del columns[-1]
    # Now we can set this list as the dataframe's columns names
    house_data.columns = columns
    
    return house_data

convert_housing_data_to_quarters()

   

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        101533.333333  104566.666667  108366.666667   
        Casper        89233.333333   89600.000000   89733.333333   
        Cheyenne     116866.666667  120033.333333  121533.333333   
        Evansville   128033.333333  128766.666667  130833.333333   
        Pine Bluffs   93733.333333   95066.666667   94633.333333   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        113000.000000  115833.333333  117200.000000   
        Casper        93166.666667   95500.000000   97633.333333   
        Cheyenne     123633.333333  125533.333333  126300.000000   
        Evansville   132066.666667  130566.666667  131433.333333   
        Pine Bluffs   98066.666667  103233.333333  104600.000000   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        117800.000000  117633.333333  117333.333333   
        Casper        99433.333333  100633.333333  101733.333333   
        Cheyenne     126466.666667  128133.333333  128466.666667   
        Evansville   132400.000000  133466.666667  133300.000000   
        Pine Bluffs  106500.000000  104066.666667  102233.333333   

                            2002q2  ...         2014q2         2014q3  \
State   RegionName                  ...                                 
Alabama Adamsville    73133.333333  ...   77066.666667   75966.666667   
        Alabaster    128000.000000  ...  147133.333333  147633.333333   
        Albertville   76366.666667  ...   84033.333333   84766.666667   
        Arab          87700.000000  ...  113366.666667  111700.000000   
        Ardmore                NaN  ...  140533.333333  139566.666667   
...                            ...  ...            ...            ...   
Wyoming Burns        117233.333333  ...  168866.666667  161933.333333   
        Casper       101533.333333  ...  175766.666667  177300.000000   
        Cheyenne     129633.333333  ...  177466.666667  176733.333333   
        Evansville   131066.666667  ...  296733.333333  305666.666667   
        Pine Bluffs  103566.666667  ...  148666.666667  154366.666667   

                            2014q4         2015q1         2015q2  \
State   RegionName                                                 
Alabama Adamsville    71900.000000   71666.666667   73033.333333   
        Alabaster    148700.000000  148900.000000  149566.666667   
        Albertville   86800.000000   88466.666667   89500.000000   
        Arab         111600.000000  110166.666667  109433.333333   
        Ardmore

In [42]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # From the project formulation at the beginning:
    
    # HYPOTHESIS: University towns have their mean housing prices less affected by recessions. Run a t-test to 
    # compare the ratio of the mean price of houses both in university towns and in non uni towns the quarter before 
    # the recession starts compared to the recession bottom. (price_ratio=quarter_before_recession/recession_bottom).
    
    # The null hypothesis is: The mean housing price of University towns and Non- University towns is same .
    # Alternate Hypothesis: The mean housing price of University towns and Non- University towns is different.
    
    # --------------------------------------------------------------------------------------------------------------------------
    # Logic of the hypothesis testing explained in detail:
    # --------------------------------------------------------------------------------------------------------------------------
    # Statement hypothesis (alternate hypothesis) – University towns have their mean housing price less affected by recessions.

    # Question – How do we measure the effect of a recession on the mean housing price in university towns? 
    # Answer – Using price ratio, we measure the effect of a recession on the mean housing price in university towns

    # Mean price ratio of university towns versus mean price ratio of non-university towns:

    # SCENARIO 1: If p-value > significance threshold (say 0.01), then we cannot reject the null hypothesis of identical averages.

    # Null hypothesis is always of identical average scores –> mean [price ratio] doesn’t differ significantly across samples 
    # ‘or’ is more or less the same between university towns and non-university towns –> Not being able to reject the null 
    # hypothesis implies that it is more credible (means are same across both samples) which in turn implies that the effects 
    # of recession are more or less the same in all towns, regardless of them being university towns or non-university towns.

    # SCENARIO 2: If p-value < significance threshold (say 0.01), then we can reject the null hypothesis of identical averages.

    # Alternate hypothesis is always of different average scores –> mean [price ratio] differs significantly across samples 
    # –> Being able to reject the null hypothesis implies that it is less credible than the alternative hypothesis (which is the
    # opposite of null hypothesis; i.e. means are different across both samples) which in turn implies that the effects of 
    # recession are different for university towns and in non-university towns.

    # How do we add in the factor of ‘less effect’ or ‘more effect’?
    # Let’s say p-value indeed is less than alpha. Hence, we reject the null hypothesis –> effects of recession are different for
    # university towns and non-university towns. Then, we compare mean price ratio = mean of price ratio column for both categories
    # of towns. We find that ‘mean price ratio for university towns’ < ‘mean price ratio for non-university towns’ (comparison of 
    # 1 number to another number)

    # Recall that mean price ratio = (mean price during quarter before recession) / (mean price during recession bottom). Because 
    # mean price ratio for university towns is less than that for non-university towns, we deduce that effects of recession on 
    # mean housing price are more pronounced in non-university towns than university towns.
    # --------------------------------------------------------------------------------------------------------------------------
    
    # We have to use a few of the previous functions to work in this one, but we will assign shorter names to them
    house_data = convert_housing_data_to_quarters()
    rec_sta = get_recession_start()
    rec_bot = get_recession_bottom()
    uni_towns = get_list_of_university_towns().set_index(['State', 'RegionName']) # We set the index the same way as the house_data df
    
    # Now we are able to calculate the ratio which is defined at the beginning of the project as:
    # (price_ratio=quarter_before_recession/recession_bottom). If the Ratio is > 1 that means that the prices were higher 
    # before the recession, if it is < 1, that means the prices got higher during the recession (prices not affected by the recession, 
    # which is equivilent to a reduced market loss).
    
    # We use the get_loc attribute to find the column index, because we need to substract one to this index to get the index of
    # the quarter BEFORE the recession starts (the function get_recession_start() gives us the quarter WHEN IT STARTS, as the 
    # name states)
    bef_rec_sta_ind = house_data.columns.get_loc(rec_sta)-1
    house_data['Ratio'] = house_data.iloc[:,bef_rec_sta_ind]/house_data[rec_bot]
    # We just need the ratio for the purpose of this exercise, so we will reduce our house df to this column.
    # It is also important to note that this has to be converted to a frame because it will be considered a Series. We need it 
    # to be a Dataframe beacuse we have to merge it later with the Uni towns df.
    # We get rid of NaN too to finish this task
    house_data = house_data.iloc[:,-1].to_frame().dropna()
    
    # The price ratio obtained in this manner should now be divided into university towns and non-university towns which is 
    # then supplied into the ttest_ind function with the argument nan_policy="omit".
    # But first we need to check the cities in the house data which are uni towns, by using inner merging
    uni_ratios = house_data.merge(uni_towns, right_index=True, left_index=True, how='inner')
    # Now the ratios of those cities which are not uni
    nonuni_ratios = house_data.drop(uni_ratios.index)
    
    p_value = ttest_ind(uni_ratios.values, nonuni_ratios.values).pvalue
    
    # We state the variable different to evaluate the hypothesis testing
    if p_value < 0.01:
        different=True
    else:
        different=False
        
    # Better depending on which one is LOWER! Remember prices go up during a recession so lower is better.
    if uni_ratios.mean().values < nonuni_ratios.mean().values:
        better='university town'
    else:
        better='non-university town'
    
    return (different, p_value[0], better)

run_ttest()

(True, 0.002724063704761164, 'university town')